<img src=https://d1.awsstatic.com/logos/aws-logo-lockups/poweredbyaws/PB_AWS_logo_RGB_REV_SQ.8c88ac215fe4e441dc42865dd6962ed4f444a90d.png width="350">

# <center> Data Engineering Lab 1 </center>
<center><b> Data Architect: </b> Carlos Contreras </center>

Dataset and metadata details; i.e. column definitions, at the [USA Bureau of Transport Statistics URL](https://www.transtats.bts.gov/DL_SelectFields.asp?gnoyr_VQ=FGK&QO_fu146_anzr=b0-gvzr)

In [ ]:
import sys
from awsglue.transforms import *
from awsglue.dynamicframe import DynamicFrame
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

# Other Libs for the ETL
from datetime import datetime, timedelta
from pyspark.sql.functions import lit, to_date, max as _max, date_sub, col

In [ ]:
# Spark settings
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)

#### Parameters below, to be set as Job input parameters (not hard-coded)

In [ ]:
# Source data path. 
s3_file_full_path = 's3://us-bts-raw-euwest1-14317621-dev/flights-performance-raw-csv-sample/'

# Job params
input_last_days_to_filter = 7
s3_vertices_location = 's3://us-bts-raw-euwest1-14317621-dev/neptune-flight-stats-graph-data/vertices/'
s3_edges_location = 's3://us-bts-raw-euwest1-14317621-dev/neptune-flight-stats-graph-data/edges/'

In [ ]:
# DF
df_flight_stats_raw = glueContext.create_dynamic_frame.from_options(
    's3',
    {'paths': [s3_file_full_path]},
    'csv',
    {'withHeader': True}).toDF()

In [ ]:
# Temporary print dataset / dataFrame Schema
df_flight_stats_raw.printSchema()

**Performance note**: To improve the execution plan, avoid PRINTS (e.g. SHOW()) like the one below, 
                    on real Spark Jobs (i.e. not like in Jupyter). 

In [ ]:
# Temporary print dataset sample
df_flight_stats_raw.select('FL_DATE','ORIGIN_AIRPORT_ID','ORIGIN_CITY_NAME','DEST_AIRPORT_ID','DEST_CITY_NAME').show()

### Get max Flight date from S3 partition, to limit the last N days

<br>

- This might not be necessary, since you might ingest the entire incremental new dataset

<br>

In [ ]:
# FL_DATE cast data type
df_flight_stats = df_flight_stats_raw.withColumn('FL_DATE',to_date(df_flight_stats_raw.FL_DATE, 'yyyy-MM-dd'))

In [ ]:
# Get max date
flight_date_max = df_flight_stats.select(_max('FL_DATE')).first()[0]

In [ ]:
# Days to store in graph
last_days_to_filter = timedelta(input_last_days_to_filter)
flight_date_max_minus_n_days = flight_date_max-last_days_to_filter

In [ ]:
# Collect last 7 days
df_flight_stats = df_flight_stats.where(col('FL_DATE') >= lit(str(flight_date_max_minus_n_days)))

---

## Creating Edges File, for a Time-Series graph

Pseudo-columns for Edges file:

- ORIGIN_AIRPORT_ID to be converted as ~from
- DEST_AIRPORT_ID to be converted as ~to

Label columns: 

- String **flight_connection** to be added as ~label

Time-series columns: 

- DISTANCE 
- ORIGIN_CITY_NAME
- DEST_CITY_NAME
- MKT_CARRIER_FL_NUM
- MKT_UNIQUE_CARRIER
- FL_DATE
- DEP_DELAY
- DEP_DEL15
- ARR_DELAY
- ARR_DEL15
- CANCELLED
- ACTUAL_ELAPSED_TIME
- WEATHER_DELAY
- SECURITY_DELAY

In [ ]:
# Collect columns we require.
df_edges_file = df_flight_stats.select('ORIGIN_AIRPORT_ID','ORIGIN','DEST_AIRPORT_ID','DEST','ORIGIN_CITY_NAME',
                                       'DEST_CITY_NAME',
                                       'MKT_UNIQUE_CARRIER','MKT_CARRIER_FL_NUM','DISTANCE','FL_DATE',
                                       'DEP_DELAY','DEP_DEL15','ARR_DELAY','ARR_DEL15','CANCELLED',
                                       'ACTUAL_ELAPSED_TIME','WEATHER_DELAY','SECURITY_DELAY')

In [ ]:
# Adding new column with label "route", defined as Integer
df_edges_file = df_edges_file.withColumn('~label', lit('flight_connection'))

In [ ]:
# Rename columns, to make it Neptune-Gremlin-compatible and more human-readable
df_edges_file = df_edges_file.withColumnRenamed('ORIGIN_AIRPORT_ID','~from')\
                        .withColumnRenamed('DEST_AIRPORT_ID','~to')\
                        .withColumnRenamed('ORIGIN','ORIGIN_CODE:string')\
                        .withColumnRenamed('DEST','DESTINATION_CODE:string')\
                        .withColumnRenamed('ORIGIN_CITY_NAME','ORIGIN_CITY_NAME:string')\
                        .withColumnRenamed('DEST_CITY_NAME','DESTINATION_CITY_NAME:string')\
                        .withColumnRenamed('MKT_UNIQUE_CARRIER','CARRIER_NAME:int')\
                        .withColumnRenamed('MKT_CARRIER_FL_NUM','CARRIER_FLIGHT_NUM:int')\
                        .withColumnRenamed('DISTANCE','DISTANCE:int')\
                        .withColumnRenamed('FL_DATE','FLIGHT_DATE:date')\
                        .withColumnRenamed('DEP_DELAY','DEPARTURE_DELAY:int')\
                        .withColumnRenamed('DEP_DEL15','DEPARTURE_DELAY_15MIN:int')\
                        .withColumnRenamed('ARR_DELAY','ARRIVAL_DELAY:int')\
                        .withColumnRenamed('ARR_DEL15','ARRIVAL_DELAY_15MIN:int')\
                        .withColumnRenamed('CANCELLED','CANCELLED:int')\
                        .withColumnRenamed('ACTUAL_ELAPSED_TIME','ACTUAL_ELAPSED_TIME:int')\
                        .withColumnRenamed('WEATHER_DELAY','WEATHER_DELAY:int')\
                        .withColumnRenamed('SECURITY_DELAY','SECURITY_DELAY:int')

In [ ]:
# Print sample
df_edges_file.select('~from','~to','~label','FLIGHT_DATE:date').show()

#### Writing data to Amazon S3

In [ ]:
# Writing data to Amazon S3
df_edges_file = df_edges_file.repartition(1)
df_edges_file.write.mode("append").format("csv").option("header", "true").save(s3_edges_location)

---

## Creating Vertices File

In [ ]:
# Collect columns we require.
df_vertices_file = df_flight_stats.\
                          select(col('ORIGIN_AIRPORT_ID').alias('~id'),
                                 col('ORIGIN').alias('ORIGIN_CODE:string'),
                                 col('ORIGIN_CITY_NAME').alias('CITY_NAME:string'))\
                   .union(df_flight_stats.\
                          select(col('DEST_AIRPORT_ID').alias('~id'),
                                 col('DEST').alias('DESTINATION_CODE:string'),
                                 col('DEST_CITY_NAME').alias('CITY_NAME:string')))\
                   .distinct()

In [ ]:
# Adding new column with label "route", defined as Integer
df_vertices_file = df_vertices_file.withColumn('~label', lit('airport'))

In [ ]:
df_vertices_file.show()

#### Writing data to Amazon S3

In [ ]:
# Writing data to Amazon S3, to a single file
df_vertices_file = df_vertices_file.repartition(1)
df_vertices_file.write.mode("append").format("csv").option("header", "true").save(s3_vertices_location)